<a href="https://colab.research.google.com/github/shinebalance/DL_notes/blob/master/DeepLearningwithPython/5_1_CNN(tf2_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **(TF2.0学習)5.1 CNN**
---

## 【目的】
* 以下書籍,第5章の内容をTF2.0のsubclassingで整理して学習するnotebook  

<div align="center">
<img src="https://book.mynavi.jp/files/topics/90124_ext_06_0.jpg" frameborder="0" width="110" height="125" scrolling="yes"></img></div>  
https://book.mynavi.jp/ec/products/detail/id=90124  

## 【お品書き】
 1. CNNのしくみ(5.1)  

> 今回追加で参考にしたもの  
> https://note.nkmk.me/python-tensorflow-keras-basics/  
> https://www.atmarkit.co.jp/ait/articles/2002/27/news021.html

---


# Kerasのインポート

In [1]:
# 2.0をColabで実行する際に必要らしい
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

# enable eager execution...2.0が導入できていればDefaultでEnable
# tf.enable_eager_execution()
print(tf.executing_eagerly())

2.1.0
True


---

# 1)CNNのしくみ

## MNIST分類を行うシンプルなCNNモデル……を、Functional APIで書くと

In [0]:
inputs = tf.keras.Input(shape=(28, 28, 1))

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

'''
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())#FlattenよりもGlobalAveragePool2D()がいいらしい
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))#分類なので最後はrelu+Softmax
'''

# モデル形状の確認
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
______________________________________________

## Subclassing APIで実装すると(本命)

In [3]:
# Subclassing API
class SimpleCNN_sc(tf.keras.Model):
    def __init__(self):
        super(SimpleCNN_sc, self).__init__()
        self.cn1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
        self.max1 = tf.keras.layers.MaxPooling2D((2, 2))
        self.cn2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self.max2 = tf.keras.layers.MaxPooling2D((2, 2))
        self.cn3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False):
        x = self.cn1(x)
        x = self.max1(x)
        x = self.cn2(x)
        x = self.max2(x)
        x = self.cn3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

model = SimpleCNN_sc()
model.build((None, 28, 28, 1))

'''
inputs = tf.keras.Input(shape=(28, 28, 1))

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
'''

# モデル形状の確認
## Output shapeは表示されない
# model.summary()

"\ninputs = tf.keras.Input(shape=(28, 28, 1))\n\nx = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)\nx = tf.keras.layers.MaxPooling2D((2, 2))(x)\nx = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)\nx = tf.keras.layers.MaxPooling2D((2, 2))(x)\nx = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)\nx = tf.keras.layers.Flatten()(x)\nx = tf.keras.layers.Dense(64, activation='relu')(x)\noutputs = tf.keras.layers.Dense(10, activation='softmax')(x)\n\nmodel = tf.keras.Model(inputs=inputs, outputs=outputs)\n"

## 前処理を実施してモデルの訓練を実施……を、model.fit()ではなく自力で回す

> memo  
> tutorialだとone-hot encoding せずにsparse categorical entropyを使ってる  
> https://www.tensorflow.org/tutorials/quickstart/advanced?hl=ja

### 試行1: metric:Accuracy
* 元ネタのKeras実装を文言通りに実行した結果

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 28*28に成形して正規化
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# カテゴリ値に変更
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

'''
# all keras ver
# オプティマイザ, 損失関数, メトリックを決めて学習実行
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)
'''

# Optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
# loss function
loss_fn = tf.keras.losses.categorical_crossentropy
# metric... need ()
metrics_acc = tf.keras.metrics.Accuracy()
# metrics_acc = tf.keras.metrics.CategoricalCrossentropy()

# set status
epochs = 5
batch_size = 64
'''
print(train_images.shape)
print(test_images.shape)
'''
# tf.Data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(64)

In [0]:
print(train_images.shape)
print(train_images.dtype)
print(train_labels.dtype)
print(train_labels.shape)
# one-hot encodingされていることを確認

(60000, 28, 28, 1)
float32
float32
(60000, 10)


In [0]:
# Iterate
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch+1,))

  # Inside epoch(per batch size)
  for step, (train_images, train_labels) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      _y = model(train_images)
      # loss
      loss = loss_fn(train_labels, _y)
      loss += sum(model.losses)  # pile up loss

    # compute grads
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    #print(train_labels[step])
    #print(_y[step])
    metrics_acc(train_labels, _y)

    if step % 512 == 0:
      #print('step %s: mean loss = %s' % (step, metrics_acc.result()))
      template = 'Step {}, metrics_acc: {} %'
      print(template.format(step, metrics_acc.result() * 100))
    
    metrics_acc.reset_states()

Start of epoch 1
Step 0, metrics_acc: 0.78125 %
Step 512, metrics_acc: 82.5 %
Start of epoch 2
Step 0, metrics_acc: 81.875 %
Step 512, metrics_acc: 83.125 %
Start of epoch 3
Step 0, metrics_acc: 83.125 %
Step 512, metrics_acc: 82.1875 %
Start of epoch 4
Step 0, metrics_acc: 82.1875 %
Step 512, metrics_acc: 82.1875 %
Start of epoch 5
Step 0, metrics_acc: 82.1875 %
Step 512, metrics_acc: 83.4375 %


#### 検証データの精度を確認

In [0]:
test_metrics_acc = tf.keras.metrics.Accuracy()

# Run a validation loop at the end of each epoch.
for test_images, test_labels in test_dataset:
  _y_pred = model(test_images)
  test_metrics_acc(test_labels, _y_pred)

val_acc = test_metrics_acc.result()
test_metrics_acc.reset_states()
print('Validation acc: %s' % (float(val_acc),))

Validation acc: 0.822700023651123


> While our densely-connected network from Chapter 2 had a test accuracy of 97.8%, our basic convnet has a test accuracy of 99.3%: we decreased our error rate by 68% (relative). Not bad! 

全結合器が97.8%なので、その値超える結果なら上々だね！ by Francois

精度が落ちている？？？

### 試行2: metric:metrics.SparseCategoricalAccuracy
* 公式チュートリアルの組み合わせをやってみる

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 28*28に成形して正規化
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.0

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255.0


# カテゴリ値に変更…を行わない
# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

# float32化しておく
# train_labels = train_labels.astype('float32')
# test_labels  = test_labels.astype('float32')

# Optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
# loss function...SparseCategoricalCrossentropy need()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# metric...Accuracy()>>> need ()
metrics_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# set status
epochs = 5
batch_size = 64
'''
print(train_images.shape)
print(test_images.shape)
'''
# tf.Data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(64)

Using TensorFlow backend.


In [5]:
print(train_images.shape)
print(train_images.dtype)
print(train_labels.dtype)
print(train_labels.shape)
# one-hot encodingされていないことを確認

(60000, 28, 28, 1)
float32
uint8
(60000,)


In [17]:
# timestamp
import time

# Iterate
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch+1,))
  _start = time.time()

  # Inside epoch(per batch size)
  for step, (train_images, train_labels) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      _y = model(train_images)
      # loss
      loss = loss_fn(train_labels, _y)
      loss += sum(model.losses)  # pile up loss

    # compute grads
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    metrics_acc(train_labels, _y)

    if step % 512 == 0:
      #print('step %s: mean loss = %s' % (step, metrics_acc.result()))
      template = 'Step {}, metrics_acc: {} %'
      print(template.format(step, metrics_acc.result() * 100))
    
    metrics_acc.reset_states()

_end = time.time() - _start
print("secs:", _end)

Start of epoch 1
Step 0, metrics_acc: 98.4375 %
Step 512, metrics_acc: 100.0 %
Start of epoch 2
Step 0, metrics_acc: 100.0 %
Step 512, metrics_acc: 100.0 %
Start of epoch 3
Step 0, metrics_acc: 100.0 %
Step 512, metrics_acc: 100.0 %
Start of epoch 4
Step 0, metrics_acc: 100.0 %
Step 512, metrics_acc: 100.0 %
Start of epoch 5
Step 0, metrics_acc: 100.0 %
Step 512, metrics_acc: 100.0 %
secs: 16.93676447868347


#### 検証

In [18]:
test_metrics_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# Run a validation loop at the end of each epoch.
for test_images, test_labels in test_dataset:
  _start = time.time()
  _y_pred = model(test_images)
  test_metrics_acc(test_labels, _y_pred)

val_acc = test_metrics_acc.result()
test_metrics_acc.reset_states()
print('Validation acc: %s' % (float(val_acc),))

_end = time.time() - _start
print("secs:", _end)

Validation acc: 0.9918000102043152
secs: 0.005369424819946289


### 試行3:@tf.functionを用いた高速化を試す

In [5]:
# timestamp
import time

# fill train cycle in def
@tf.function
def train_cycle(train_image, train_labels):
  with tf.GradientTape() as tape:
    _y = model(train_images)
    loss = loss_fn(train_labels, _y)
    loss += sum(model.losses)  # pile up loss
  # compute grads
  grads = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  metrics_acc(train_labels, _y)


# Iterate
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch+1,))
  _start = time.time()

  # Inside epoch(per batch size)
  for step, (train_images, train_labels) in enumerate(train_dataset):
    train_cycle(train_images, train_labels)

    if step % 512 == 0:
      #print('step %s: mean loss = %s' % (step, metrics_acc.result()))
      template = 'Step {}, metrics_acc: {} %'
      print(template.format(step, metrics_acc.result() * 100))
    
    metrics_acc.reset_states()

_end = time.time() - _start
print("secs:", _end)

Start of epoch 1
ERROR! Session/line number was not unique in database. History logging moved to new session 65
Step 0, metrics_acc: 10.9375 %
Step 512, metrics_acc: 12.5 %
Start of epoch 2
Step 0, metrics_acc: 6.25 %
Step 512, metrics_acc: 14.0625 %
Start of epoch 3
Step 0, metrics_acc: 12.5 %
Step 512, metrics_acc: 7.8125 %
Start of epoch 4
Step 0, metrics_acc: 9.375 %
Step 512, metrics_acc: 10.9375 %
Start of epoch 5
Step 0, metrics_acc: 17.1875 %
Step 512, metrics_acc: 9.375 %
secs: 3.372530698776245


精度は異様に落ちたが速度は段違いに早い

<br>
<br>
<br>
<br>
<br>
  
---

編集前のnoteではこのあと5.2 ,3 ,4を1つのnoteにまとめていたが、上記の通り内容が増えたため別のnoteでtf2.0化を行う  
<br>

**EOF**